# Perbandingan Metode K-Nearest Neighbor dan Random Forest dalam Klasifikasi Status Stunting Anak Berdasarkan Data Antropometri

**Tugas Besar Praktikum Machine Learning**

---

## Latar Belakang

**Stunting** adalah kondisi gagal tumbuh pada anak akibat kekurangan gizi kronis terutama pada 1000 hari pertama kehidupan. Klasifikasi status stunting sangat penting untuk deteksi dini dan intervensi yang tepat.

## Tujuan

Notebook ini bertujuan untuk membandingkan performa dua algoritma machine learning:
- **KNN (K-Nearest Neighbor)** - sebagai Baseline Model
- **Random Forest** - sebagai Advanced Model

dalam mengklasifikasikan status stunting anak berdasarkan data antropometri.

## Dataset

Dataset yang digunakan berisi 100.000 data dengan fitur:
- Jenis Kelamin
- Umur (bulan)
- Tinggi Badan (cm)
- Berat Badan (kg)
- **Stunting** (Target Variable)

---
## 1. Setup Google Colab & Load Data

Pada tahap ini, kita akan:
1. **Mount Google Drive** - Menghubungkan Google Colab dengan Google Drive untuk mengakses dataset
2. **Import Libraries** - Mengimpor semua library yang diperlukan untuk analisis dan pemodelan

### 1.1 Mount Google Drive

Kode berikut digunakan untuk menghubungkan Google Colab dengan akun Google Drive Anda. Setelah dijalankan, Anda akan diminta untuk memberikan izin akses.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

### 1.2 Import Libraries

Library yang digunakan:
- **pandas & numpy**: Manipulasi dan analisis data
- **matplotlib & seaborn**: Visualisasi data
- **scikit-learn**: Preprocessing, modeling, dan evaluasi
- **joblib**: Menyimpan dan memuat model

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
import os
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report,
                             roc_curve, auc)
from sklearn.preprocessing import label_binarize

print("✅ Semua library berhasil diimport!")

### 1.3 Load Dataset

**⚠️ PENTING:** Sesuaikan `DATA_PATH` dengan lokasi file dataset di Google Drive Anda.

Folder `models` akan dibuat otomatis untuk menyimpan model yang sudah ditraining.

In [ ]:
# ⚠️ PENTING: Sesuaikan path ini dengan lokasi file di Google Drive Anda!
DATA_PATH = '/content/drive/MyDrive/TB Machine Learning/stunting_wasting_dataset.csv'
MODEL_PATH = '/content/drive/MyDrive/TB Machine Learning/models/'

# Buat folder models jika belum ada
os.makedirs(MODEL_PATH, exist_ok=True)

df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset berhasil dimuat!")
print(f"📊 Shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")

### 1.4 Preview Data

Melihat struktur dan sampel data untuk memahami karakteristik dataset.

In [ ]:
# Menampilkan 10 data pertama
df.head(10)

In [ ]:
# Info dataset
df.info()

---
## 2. Exploratory Data Analysis (EDA)

**EDA** adalah proses penting untuk memahami karakteristik data sebelum pemodelan. Pada tahap ini kita akan:
1. Menganalisis statistik deskriptif
2. Memeriksa missing values
3. Memvisualisasikan distribusi data
4. Menganalisis korelasi antar fitur

### 2.1 Statistik Deskriptif

Statistik deskriptif memberikan gambaran umum tentang data numerik seperti mean, median, standar deviasi, nilai minimum dan maksimum.

In [ ]:
# Statistik Deskriptif
print("=" * 60)
print("STATISTIK DESKRIPTIF")
print("=" * 60)
df.describe()

### 2.2 Cek Missing Values

Missing values dapat mempengaruhi performa model. Penting untuk mengidentifikasi dan menangani data yang hilang sebelum pemodelan.

In [ ]:
# Cek Missing Values
print("=" * 60)
print("MISSING VALUES")
print("=" * 60)
missing = df.isnull().sum()
print(missing)
print(f"\nTotal missing values: {missing.sum()}")

### 2.3 Distribusi Target Variable (Stunting)

Penting untuk memahami distribusi kelas target untuk mengetahui apakah data seimbang atau tidak. Data yang tidak seimbang dapat mempengaruhi performa model.

In [ ]:
# Distribusi Target Variable (Stunting)
plt.figure(figsize=(10, 6))
colors = ['#FF6B6B', '#FFA94D', '#69DB7C', '#4DABF7']
stunting_counts = df['Stunting'].value_counts()
plt.pie(stunting_counts, labels=stunting_counts.index, autopct='%1.1f%%', 
        colors=colors, explode=[0.02]*len(stunting_counts), shadow=True)
plt.title('Distribusi Status Stunting', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nDistribusi Stunting:")
print(stunting_counts)

### 2.4 Distribusi Jenis Kelamin

Melihat proporsi data berdasarkan jenis kelamin untuk memastikan representasi yang seimbang.

In [ ]:
# Distribusi Jenis Kelamin
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Jenis Kelamin', palette='Set2')
plt.title('Distribusi Jenis Kelamin', fontsize=14, fontweight='bold')
plt.xlabel('Jenis Kelamin')
plt.ylabel('Jumlah')
plt.tight_layout()
plt.show()

### 2.5 Distribusi Fitur Numerik

Memvisualisasikan distribusi fitur numerik (Umur, Tinggi Badan, Berat Badan) menggunakan histogram untuk melihat pola dan outlier.

In [ ]:
# Distribusi Fitur Numerik
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

features_numeric = ['Umur (bulan)', 'Tinggi Badan (cm)', 'Berat Badan (kg)']
colors = ['#4ECDC4', '#FF6B6B', '#45B7D1']

for idx, (feature, color) in enumerate(zip(features_numeric, colors)):
    axes[idx].hist(df[feature], bins=30, color=color, edgecolor='white', alpha=0.8)
    axes[idx].set_title(f'Distribusi {feature}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Frekuensi')

plt.tight_layout()
plt.show()

### 2.6 Boxplot: Fitur Numerik vs Status Stunting

Boxplot membantu melihat hubungan antara fitur numerik dengan status stunting, serta mengidentifikasi outlier.

In [ ]:
# Boxplot berdasarkan Status Stunting
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, feature in enumerate(features_numeric):
    sns.boxplot(data=df, x='Stunting', y=feature, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{feature} vs Stunting', fontsize=12, fontweight='bold')
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 2.7 Heatmap Korelasi

Heatmap korelasi menunjukkan hubungan linear antar fitur. Nilai mendekati 1 atau -1 menunjukkan korelasi kuat, sedangkan nilai mendekati 0 menunjukkan korelasi lemah.

In [ ]:
# Heatmap Korelasi
plt.figure(figsize=(8, 6))
df_corr = df.copy()
df_corr['Jenis Kelamin'] = LabelEncoder().fit_transform(df_corr['Jenis Kelamin'])
df_corr['Stunting'] = LabelEncoder().fit_transform(df_corr['Stunting'])
df_corr['Wasting'] = LabelEncoder().fit_transform(df_corr['Wasting'])

correlation = df_corr.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', linewidths=0.5)
plt.title('Heatmap Korelasi Antar Fitur', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 3. Data Preprocessing

**Preprocessing** adalah tahap persiapan data sebelum dimasukkan ke model. Tahapan yang dilakukan:
1. Memisahkan fitur (X) dan target (y)
2. Label Encoding untuk fitur kategorik
3. Feature Scaling menggunakan StandardScaler
4. Train-Test Split

### 3.1 Memisahkan Fitur dan Target

Memisahkan kolom yang akan digunakan sebagai fitur input (X) dan kolom target yang akan diprediksi (y).

In [ ]:
# Memisahkan fitur dan target
X = df[['Jenis Kelamin', 'Umur (bulan)', 'Tinggi Badan (cm)', 'Berat Badan (kg)']].copy()
y = df['Stunting'].copy()

print("Fitur (X):")
print(X.head())
print(f"\nShape X: {X.shape}")
print(f"\nTarget (y) unique values: {y.unique()}")

### 3.2 Label Encoding

**Label Encoding** mengubah data kategorik menjadi numerik agar dapat diproses oleh algoritma machine learning.
- Jenis Kelamin: Laki-laki → 0, Perempuan → 1
- Stunting: Severely Stunted → 0, Stunted → 1, Normal → 2, Tall → 3

In [ ]:
# Label Encoding untuk Jenis Kelamin
le_gender = LabelEncoder()
X['Jenis Kelamin'] = le_gender.fit_transform(X['Jenis Kelamin'])
print(f"Mapping Jenis Kelamin: {dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_)))}")

# Label Encoding untuk Target (Stunting)
le_stunting = LabelEncoder()
y_encoded = le_stunting.fit_transform(y)
print(f"Mapping Stunting: {dict(zip(le_stunting.classes_, le_stunting.transform(le_stunting.classes_)))}")

### 3.3 Feature Scaling

**StandardScaler** menstandarisasi fitur dengan menghilangkan mean dan menskalakan ke unit variance.

$$z = \frac{x - \mu}{\sigma}$$

Ini penting terutama untuk algoritma KNN yang sensitif terhadap skala data.

In [ ]:
# Feature Scaling menggunakan StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print("Data setelah scaling:")
print(X_scaled.describe())

### 3.4 Train-Test Split

Data dibagi menjadi:
- **Training set (80%)**: Untuk melatih model
- **Testing set (20%)**: Untuk mengevaluasi performa model

Parameter `stratify=y_encoded` memastikan proporsi kelas yang seimbang di kedua set.

In [ ]:
# Train-Test Split (80:20, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Reset index untuk menghindari masalah indexing
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

print(f"✅ Data berhasil dibagi!")
print(f"📊 Training set: {X_train.shape[0]} samples")
print(f"📊 Testing set: {X_test.shape[0]} samples")

---
## 4. Model KNN (Baseline)

**K-Nearest Neighbor (KNN)** adalah algoritma yang mengklasifikasikan data berdasarkan kedekatan dengan tetangga terdekatnya.

### Cara Kerja KNN:
1. Hitung jarak antara data baru dengan semua data training
2. Ambil K tetangga terdekat
3. Lakukan voting mayoritas untuk menentukan kelas

### Kelebihan:
- Mudah dipahami dan diimplementasikan
- Tidak memerlukan training time

### Kekurangan:
- Lambat pada dataset besar
- Sensitif terhadap skala data dan outlier

### 4.1 Hyperparameter Tuning - Mencari Nilai K Optimal

Nilai K menentukan jumlah tetangga yang dipertimbangkan. K yang terlalu kecil dapat menyebabkan overfitting, sedangkan K yang terlalu besar dapat menyebabkan underfitting.

In [ ]:
# Hyperparameter Tuning - Mencari nilai K optimal
k_values = range(3, 21, 2)
train_scores = []
test_scores = []

print("=" * 60)
print("HYPERPARAMETER TUNING KNN")
print("=" * 60)

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    train_acc = knn.score(X_train, y_train)
    test_acc = knn.score(X_test, y_test)
    
    train_scores.append(train_acc)
    test_scores.append(test_acc)
    
    print(f"K = {k:2d} | Train Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")

### 4.2 Visualisasi Pengaruh Nilai K

Grafik ini menunjukkan bagaimana akurasi berubah seiring perubahan nilai K.

In [ ]:
# Visualisasi K vs Accuracy
plt.figure(figsize=(10, 6))
plt.plot(k_values, train_scores, 'b-o', label='Training Accuracy', linewidth=2, markersize=8)
plt.plot(k_values, test_scores, 'r-s', label='Testing Accuracy', linewidth=2, markersize=8)
plt.xlabel('Nilai K', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('KNN: Pengaruh Nilai K terhadap Accuracy', fontsize=14, fontweight='bold')
plt.xticks(k_values)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

best_k = k_values[np.argmax(test_scores)]
print(f"\n🏆 Nilai K optimal: {best_k} dengan Test Accuracy: {max(test_scores):.4f}")

### 4.3 Training Model KNN dengan K Optimal

In [ ]:
# Training Model KNN dengan K optimal
knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(X_train, y_train)

y_pred_knn = knn_model.predict(X_test)
print(f"✅ Model KNN berhasil ditraining dengan K = {best_k}")

### 4.4 Evaluasi Model KNN

Metrik evaluasi yang digunakan:
- **Accuracy**: Proporsi prediksi yang benar
- **Precision**: Ketepatan prediksi positif
- **Recall**: Kemampuan mendeteksi kelas positif
- **F1-Score**: Harmonic mean dari precision dan recall

In [ ]:
# Evaluasi Model KNN
print("=" * 60)
print("EVALUASI MODEL KNN (BASELINE)")
print("=" * 60)

knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_precision = precision_score(y_test, y_pred_knn, average='weighted')
knn_recall = recall_score(y_test, y_pred_knn, average='weighted')
knn_f1 = f1_score(y_test, y_pred_knn, average='weighted')

print(f"Accuracy : {knn_accuracy:.4f}")
print(f"Precision: {knn_precision:.4f}")
print(f"Recall   : {knn_recall:.4f}")
print(f"F1-Score : {knn_f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_knn, target_names=le_stunting.classes_))

### 4.5 Confusion Matrix KNN

Confusion Matrix menunjukkan jumlah prediksi yang benar dan salah untuk setiap kelas.

In [ ]:
# Confusion Matrix KNN
plt.figure(figsize=(8, 6))
cm_knn = confusion_matrix(y_test, y_pred_knn)
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_stunting.classes_, yticklabels=le_stunting.classes_)
plt.title('Confusion Matrix - KNN', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

### 4.6 ROC Curve KNN

**ROC Curve (Receiver Operating Characteristic)** menunjukkan trade-off antara True Positive Rate dan False Positive Rate pada berbagai threshold. Area Under Curve (AUC) mengukur seberapa baik model dalam membedakan kelas.

- AUC = 1.0: Model sempurna
- AUC = 0.5: Model sama dengan random guessing
- AUC < 0.5: Model lebih buruk dari random guessing

In [ ]:
# ROC Curve untuk KNN (One-vs-Rest)
y_bin = label_binarize(y_test, classes=range(len(le_stunting.classes_)))
y_proba_knn = knn_model.predict_proba(X_test)

plt.figure(figsize=(10, 8))
colors = ['#FF6B6B', '#FFA94D', '#69DB7C', '#4DABF7']

for i, (color, class_name) in enumerate(zip(colors, le_stunting.classes_)):
    fpr, tpr, _ = roc_curve(y_bin[:, i], y_proba_knn[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2, label=f'{class_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - KNN', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 5. Model Random Forest (Advanced)

**Random Forest** adalah algoritma ensemble yang menggabungkan banyak Decision Tree untuk menghasilkan prediksi yang lebih akurat.

### Cara Kerja:
1. Buat banyak Decision Tree dengan subset data yang berbeda (Bootstrap)
2. Setiap tree melakukan prediksi secara independen
3. Hasil akhir ditentukan oleh voting mayoritas

### Kelebihan:
- Robust terhadap overfitting
- Dapat menangani hubungan non-linear
- Memberikan informasi feature importance

### Kekurangan:
- Lebih kompleks dan membutuhkan resource lebih besar
- Training time lebih lama

### 5.1 Hyperparameter Tuning dengan GridSearchCV

**GridSearchCV** mencoba semua kombinasi hyperparameter untuk menemukan yang terbaik.

Parameter yang di-tune:
- **n_estimators**: Jumlah tree dalam forest
- **max_depth**: Kedalaman maksimum tree
- **min_samples_split**: Minimum sampel untuk split node

In [ ]:
# Hyperparameter Tuning menggunakan GridSearchCV
print("=" * 60)
print("HYPERPARAMETER TUNING RANDOM FOREST")
print("=" * 60)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

subset_size = min(10000, len(X_train))
np.random.seed(42)
subset_indices = np.random.choice(len(X_train), size=subset_size, replace=False)

X_train_subset = X_train.iloc[subset_indices]
y_train_subset = y_train[subset_indices]

print(f"📊 Subset size: {len(X_train_subset)} samples")
print("⏳ Melakukan Grid Search (mungkin membutuhkan beberapa menit)...")

grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train_subset, y_train_subset)

print(f"\n🏆 Best Parameters: {grid_search.best_params_}")
print(f"🏆 Best Cross-Validation Score: {grid_search.best_score_:.4f}")

### 5.2 Training Model Random Forest dengan Parameter Terbaik

In [ ]:
# Training Model Random Forest dengan parameter terbaik
rf_model = RandomForestClassifier(
    n_estimators=grid_search.best_params_['n_estimators'],
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_split=grid_search.best_params_['min_samples_split'],
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print(f"✅ Model Random Forest berhasil ditraining!")

### 5.3 Evaluasi Model Random Forest

In [ ]:
# Evaluasi Model Random Forest
print("=" * 60)
print("EVALUASI MODEL RANDOM FOREST (ADVANCED)")
print("=" * 60)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf, average='weighted')
rf_recall = recall_score(y_test, y_pred_rf, average='weighted')
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Accuracy : {rf_accuracy:.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall   : {rf_recall:.4f}")
print(f"F1-Score : {rf_f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=le_stunting.classes_))

### 5.4 Confusion Matrix Random Forest

In [ ]:
# Confusion Matrix Random Forest
plt.figure(figsize=(8, 6))
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', 
            xticklabels=le_stunting.classes_, yticklabels=le_stunting.classes_)
plt.title('Confusion Matrix - Random Forest', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

### 5.5 ROC Curve Random Forest

ROC Curve untuk Random Forest menunjukkan performa model dalam membedakan setiap kelas stunting. AUC yang lebih tinggi menunjukkan model yang lebih baik.

In [ ]:
# ROC Curve untuk Random Forest (One-vs-Rest)
y_proba_rf = rf_model.predict_proba(X_test)

plt.figure(figsize=(10, 8))
colors = ['#FF6B6B', '#FFA94D', '#69DB7C', '#4DABF7']

for i, (color, class_name) in enumerate(zip(colors, le_stunting.classes_)):
    fpr, tpr, _ = roc_curve(y_bin[:, i], y_proba_rf[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2, label=f'{class_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Random Forest', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 6. Evaluasi dan Perbandingan Model

Pada bagian ini, kita akan membandingkan performa kedua model menggunakan:
1. **Cross-Validation**: Untuk evaluasi yang lebih robust
2. **Tabel Perbandingan**: Melihat metrik secara side-by-side
3. **Visualisasi**: Grafik perbandingan yang mudah dipahami

### 6.1 Cross-Validation (5-Fold)

**Cross-Validation** membagi data menjadi 5 bagian, melatih model pada 4 bagian dan menguji pada 1 bagian, diulang 5 kali. Ini memberikan estimasi performa yang lebih reliable.

In [ ]:
# Cross-Validation untuk kedua model
print("=" * 60)
print("CROSS-VALIDATION (5-Fold)")
print("=" * 60)

cv_size = min(20000, len(X_scaled))
np.random.seed(42)
cv_indices = np.random.choice(len(X_scaled), size=cv_size, replace=False)

X_cv = X_scaled.iloc[cv_indices].reset_index(drop=True)
y_cv = y_encoded[cv_indices]

cv_knn = cross_val_score(KNeighborsClassifier(n_neighbors=best_k), X_cv, y_cv, cv=5, scoring='accuracy')
cv_rf = cross_val_score(RandomForestClassifier(
    n_estimators=grid_search.best_params_['n_estimators'],
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_split=grid_search.best_params_['min_samples_split'],
    random_state=42, n_jobs=-1
), X_cv, y_cv, cv=5, scoring='accuracy')

print(f"KNN Cross-Validation Scores: {cv_knn}")
print(f"KNN Mean CV Score: {cv_knn.mean():.4f} (+/- {cv_knn.std()*2:.4f})")
print()
print(f"Random Forest Cross-Validation Scores: {cv_rf}")
print(f"Random Forest Mean CV Score: {cv_rf.mean():.4f} (+/- {cv_rf.std()*2:.4f})")

### 6.2 Tabel Perbandingan Metrik

In [ ]:
# Tabel Perbandingan Metrik
print("=" * 60)
print("PERBANDINGAN PERFORMA MODEL")
print("=" * 60)

comparison_data = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'CV Mean Score'],
    'KNN (Baseline)': [knn_accuracy, knn_precision, knn_recall, knn_f1, cv_knn.mean()],
    'Random Forest (Advanced)': [rf_accuracy, rf_precision, rf_recall, rf_f1, cv_rf.mean()]
}

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.set_index('Metric')

df_comparison_display = df_comparison.copy()
for col in df_comparison_display.columns:
    df_comparison_display[col] = df_comparison_display[col].apply(lambda x: f"{x:.4f} ({x*100:.2f}%)")

print(df_comparison_display)

### 6.3 Visualisasi Perbandingan

In [ ]:
# Visualisasi Perbandingan Metrik
fig, ax = plt.subplots(figsize=(12, 6))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'CV Mean Score']
x = np.arange(len(metrics))
width = 0.35

knn_scores = [knn_accuracy, knn_precision, knn_recall, knn_f1, cv_knn.mean()]
rf_scores = [rf_accuracy, rf_precision, rf_recall, rf_f1, cv_rf.mean()]

bars1 = ax.bar(x - width/2, knn_scores, width, label='KNN (Baseline)', color='#3498DB', edgecolor='white')
bars2 = ax.bar(x + width/2, rf_scores, width, label='Random Forest (Advanced)', color='#2ECC71', edgecolor='white')

ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Perbandingan Performa: KNN vs Random Forest', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend(fontsize=11)
ax.set_ylim(0, 1.1)

for bar, score in zip(bars1, knn_scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{score:.3f}', ha='center', va='bottom', fontsize=9)
for bar, score in zip(bars2, rf_scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---
## 7. Visualisasi dan Analisis

Analisis lanjutan untuk memahami model lebih dalam.

### 7.1 Feature Importance

Random Forest dapat menghitung **Feature Importance** yang menunjukkan fitur mana yang paling berpengaruh dalam prediksi.

In [ ]:
# Feature Importance dari Random Forest
plt.figure(figsize=(10, 6))
feature_names = ['Jenis Kelamin', 'Umur (bulan)', 'Tinggi Badan (cm)', 'Berat Badan (kg)']
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(feature_names)))
plt.bar(range(len(feature_names)), importances[indices], color=colors, edgecolor='white')
plt.xticks(range(len(feature_names)), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.xlabel('Fitur', fontsize=12)
plt.ylabel('Importance', fontsize=12)
plt.title('Feature Importance - Random Forest', fontsize=14, fontweight='bold')

for i, (idx, imp) in enumerate(zip(indices, importances[indices])):
    plt.text(i, imp + 0.01, f'{imp:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nFeature Importance Ranking:")
for i, idx in enumerate(indices):
    print(f"{i+1}. {feature_names[idx]}: {importances[idx]:.4f}")

### 7.2 Perbandingan Confusion Matrix

In [ ]:
# Perbandingan Confusion Matrix Side by Side
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=le_stunting.classes_, yticklabels=le_stunting.classes_)
axes[0].set_title('Confusion Matrix - KNN', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Predicted Label')
axes[0].set_ylabel('True Label')

sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=le_stunting.classes_, yticklabels=le_stunting.classes_)
axes[1].set_title('Confusion Matrix - Random Forest', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Predicted Label')
axes[1].set_ylabel('True Label')

plt.tight_layout()
plt.show()

---
## 8. Menyimpan Model untuk Deployment

Model yang sudah ditraining perlu disimpan agar dapat digunakan kembali tanpa harus melatih ulang. Kita menggunakan **joblib** untuk serialisasi model.

In [ ]:
# Menyimpan semua model dan preprocessor
joblib.dump(knn_model, f'{MODEL_PATH}knn_model.joblib')
print(f"✅ Model KNN berhasil disimpan")

joblib.dump(rf_model, f'{MODEL_PATH}rf_model.joblib')
print(f"✅ Model Random Forest berhasil disimpan")

joblib.dump(scaler, f'{MODEL_PATH}scaler.joblib')
print(f"✅ Scaler berhasil disimpan")

joblib.dump(le_gender, f'{MODEL_PATH}label_encoder_gender.joblib')
print(f"✅ Label Encoder Gender berhasil disimpan")

joblib.dump(le_stunting, f'{MODEL_PATH}label_encoder_stunting.joblib')
print(f"✅ Label Encoder Stunting berhasil disimpan")

In [ ]:
# Menyimpan informasi model
model_info = {
    'knn_best_k': best_k,
    'rf_best_params': grid_search.best_params_,
    'feature_names': feature_names,
    'stunting_classes': list(le_stunting.classes_),
    'gender_classes': list(le_gender.classes_),
    'knn_metrics': {'accuracy': knn_accuracy, 'precision': knn_precision, 'recall': knn_recall, 'f1_score': knn_f1},
    'rf_metrics': {'accuracy': rf_accuracy, 'precision': rf_precision, 'recall': rf_recall, 'f1_score': rf_f1}
}

joblib.dump(model_info, f'{MODEL_PATH}model_info.joblib')
print(f"✅ Model info berhasil disimpan")

print("\n" + "=" * 60)
print("SEMUA MODEL BERHASIL DISIMPAN!")
print("=" * 60)

---
## 9. Kesimpulan

Berikut adalah ringkasan hasil perbandingan kedua algoritma.

In [ ]:
print("=" * 60)
print("KESIMPULAN")
print("=" * 60)

print(f"""
📊 RINGKASAN HASIL PERBANDINGAN
{'=' * 60}

1. MODEL KNN (BASELINE)
   - Nilai K optimal: {best_k}
   - Accuracy: {knn_accuracy:.4f} ({knn_accuracy*100:.2f}%)
   - Precision: {knn_precision:.4f}
   - Recall: {knn_recall:.4f}
   - F1-Score: {knn_f1:.4f}

2. MODEL RANDOM FOREST (ADVANCED)
   - Parameters: {grid_search.best_params_}
   - Accuracy: {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)
   - Precision: {rf_precision:.4f}
   - Recall: {rf_recall:.4f}
   - F1-Score: {rf_f1:.4f}
""")

if rf_accuracy > knn_accuracy:
    better_model = "Random Forest"
    improvement = ((rf_accuracy - knn_accuracy) / knn_accuracy) * 100
    print(f"🏆 MODEL TERBAIK: {better_model}")
    print(f"   Peningkatan akurasi: {improvement:.2f}%")
else:
    print(f"🏆 MODEL TERBAIK: KNN")

print("""
💡 ANALISIS:

KELEBIHAN KNN:
- Mudah diimplementasikan
- Tidak memerlukan training time lama

KELEBIHAN RANDOM FOREST:
- Robust terhadap overfitting
- Memberikan feature importance
- Performa lebih baik untuk dataset besar

📌 Model siap untuk deployment ke Streamlit!
""")